In [ ]:
import chess
from data_manager import board_to_tensor, index_to_move
import torch
from model import Model
import numpy as np
import json

with open("../models/move_mapping.json", "r") as f:
    int_to_move = json.load(f)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

model = Model(len(int_to_move))
model.load_state_dict(torch.load("../models/DEFAULT_150.pth"))
model.to(device)
model.eval()

def predict_move(board: chess.Board) -> chess.Move:
    """
    Przewiduje ruch na podstawie stanu szachownicy
    """

    X_tensor = board_to_tensor(board).to(device)
    with torch.no_grad():
        logits = model(X_tensor)
    
    probabilities = torch.softmax(logits.squeeze(0), dim=0).cpu().numpy()
    sorted_indices = np.argsort(probabilities)[::-1]
    legal_moves = list(board.legal_moves)

    # Dekodujemy indeks na ruch UCI
    for idx in sorted_indices:
        move_index = int(int_to_move[str(idx)])
        move = chess.Move.from_uci(index_to_move(move_index).uci())
        if move in legal_moves:
            return move, probabilities[idx]

    return None, None

In [ ]:
def make_move(move,board: chess.Board) -> chess.Board:
    """
    Wykonuje ruch na szachownicy, komputer wykonuje ruch z predykcji i zwraca zaktualizowaną szachownicę
    """
    move = chess.Move.from_uci(move)
    if move in board.legal_moves:
        board.push(move)
        best_move, move_prob = predict_move(board)
        print(f"Best move: {best_move}, confidence: {move_prob:.4f}")
        board.push(best_move)
    return board

In [ ]:
board = chess.Board()
board

In [ ]:
make_move("e2e4", board)

In [ ]:
make_move("d2d4", board)

In [ ]:
make_move("b1c3", board)

In [ ]:
make_move("c1g5", board)

In [ ]:
make_move("g1f3", board)

In [ ]:
make_move("f1b5", board)

In [ ]:
make_move("f3e5", board)

In [ ]:
make_move("g5h6", board)